In [ ]:
!pip install gradio

In [ ]:
from typing import List, Dict, Optional
import requests

# A simple Agent base class
class Agent:
    def run(self, *args, **kwargs):
        raise NotImplementedError


class OpenRouterLLM:
    """OpenRouter LLM wrapper to call the chat completions API."""

    def __init__(self, api_key: str, model_name: str = "gpt-4o-mini"):
        self.api_key = api_key
        self.model_name = model_name
        self.endpoint = "https://openrouter.ai/api/v1/chat/completions"

    def call(
        self,
        messages: List[Dict[str, str]],
        temperature: float = 0.7,
        max_tokens: int = 512,
    ) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }
        payload = {
            "model": self.model_name,
            "messages": messages,
            "temperature": temperature,
            "max_tokens": max_tokens,
        }
        try:
            response = requests.post(self.endpoint, json=payload, headers=headers)
            response.raise_for_status()
            data = response.json()
            return data["choices"][0]["message"]["content"]
        except requests.RequestException as e:
            raise RuntimeError(f"OpenRouter API request failed: {e}")



In [ ]:

class QuestionGenerationAgent(Agent):
    """Agent to generate thoughtful questions from text."""

    def __init__(self, llm: OpenRouterLLM):
        self.llm = llm

    def run(self, text: str) -> str:
        system_prompt = "You are a helpful assistant who generates thoughtful questions."
        user_prompt = (
            f"Create 5 thoughtful questions based on the following text:\n\n{text}\n\nQuestions:"
        )
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        return self.llm.call(messages)



In [ ]:
class AnswerAgent(Agent):
    """Agent to answer questions based on a text."""

    def __init__(self, llm: OpenRouterLLM):
        self.llm = llm

    def run(self, text: str, questions: str) -> str:
        system_prompt = "You are an expert tutor who provides detailed answers."
        user_prompt = (
            f"Given the following text:\n\n{text}\n\n"
            f"Provide detailed answers to these questions:\n{questions}"
        )
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        return self.llm.call(messages)


In [ ]:
class SummaryAgent(Agent):
    """Agent to summarize text concisely."""

    def __init__(self, llm: OpenRouterLLM):
        self.llm = llm

    def run(self, text: str) -> str:
        system_prompt = "You are a concise summarizer."
        user_prompt = f"Summarize the following text in a concise paragraph:\n\n{text}"
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        return self.llm.call(messages)

In [ ]:
class LearningStudyAssistant(Agent):
    """Orchestrator Agent combining question generation, answering, and summarization."""

    def __init__(self, llm: OpenRouterLLM):
        self.question_agent = QuestionGenerationAgent(llm)
        self.answer_agent = AnswerAgent(llm)
        self.summary_agent = SummaryAgent(llm)

    def run(self, text: str) -> Dict[str, str]:
        questions = self.question_agent.run(text)
        answers = self.answer_agent.run(text, questions)
        summary = self.summary_agent.run(text)
        return {
            "questions": questions.strip(),
            "answers": answers.strip(),
            "summary": summary.strip(),
        }

In [ ]:


if __name__ == "__main__":
    API_KEY = "key"
    llm = OpenRouterLLM(API_KEY)
    assistant = LearningStudyAssistant(llm)

    input_text = """Biology Notes: The Cell and Its Functions

The cell is the basic unit of life. All living organisms are composed of cells, which carry out essential life processes. There are two main types of cells: prokaryotic and eukaryotic.

Prokaryotic cells lack a nucleus and membrane-bound organelles. Examples include bacteria and archaea.

Eukaryotic cells have a defined nucleus and various organelles. They are found in plants, animals, fungi, and protists.

Key Organelles and Their Functions:

Nucleus: Contains genetic material (DNA) and controls cellular activities.

Mitochondria: The powerhouse of the cell; site of cellular respiration and energy (ATP) production.

Ribosomes: Sites of protein synthesis.

Endoplasmic Reticulum (ER): Rough ER has ribosomes and synthesizes proteins; smooth ER synthesizes lipids.

Golgi Apparatus: Modifies, sorts, and packages proteins and lipids for secretion or use within the cell.

Lysosomes: Contain enzymes for digestion of cellular waste and foreign material.

Chloroplasts: Present in plant cells; sites of photosynthesis.

Cell Membrane: Semi-permeable membrane controlling the movement of substances in and out of the cell.

Cell Division:

Mitosis: Process by which a eukaryotic cell divides to produce two genetically identical daughter cells.

Meiosis: Specialized cell division producing gametes (sperm and egg) with half the chromosome number.

Photosynthesis and Respiration:

Photosynthesis converts light energy into chemical energy stored in glucose.

Cellular respiration breaks down glucose to produce ATP, the energy currency of the cell."""

    results = assistant.run(input_text)

    print("Questions:\n", results["questions"])
    print("\nAnswers:\n", results["answers"])
    print("\nSummary:\n", results["summary"])


Questions:
 1. How do the structural differences between prokaryotic and eukaryotic cells influence their respective functions and roles in the environment?

2. In what ways do the organelles within eukaryotic cells collaborate to maintain homeostasis and support the overall functionality of the cell?

3. How does the process of cellular respiration relate to photosynthesis, and why is this relationship essential for the survival of both plants and animals?

4. What are the implications of errors that occur during mitosis and meiosis, and how can these errors affect an organism's development or reproduction?

5. How might advancements in our understanding of cellular functions and organelle roles contribute to medical science, particularly in the treatment of diseases related to cell dysfunction?

Answers:
 ### 1. Structural Differences Between Prokaryotic and Eukaryotic Cells

Prokaryotic cells are simpler and smaller than eukaryotic cells and lack a nucleus and membrane-bound organel

In [ ]:
import gradio as gr
API_KEY = "key"
llm = OpenRouterLLM(API_KEY)
assistant = LearningStudyAssistant(llm)

def process_notes(notes):
    results = assistant.run(notes)
    questions = results["questions"]
    answers = results["answers"]
    summary = results["summary"]
    return questions, answers, summary

iface = gr.Interface(
    fn=process_notes,
    inputs=gr.Textbox(
        label="Enter your biology notes",
        placeholder="Paste your biology notes here...",
        lines=15,
    ),
    outputs=[
        gr.Textbox(label="Generated Questions"),
        gr.Textbox(label="Generated Answers"),
        gr.Textbox(label="Summary"),
    ],
    title="Biology Notes Analyzer",
    description="This tool takes in your biology notes and generates questions, answers, and a summary for better studying.",
    theme="default",
)

if __name__ == "__main__":
    iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1488920b38cb4b449f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
